<a href="https://colab.research.google.com/github/Priyo-prog/Statistics-and-Data-Science/blob/main/Preparing%20Dataset/preparing_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Preparing Dataset**

* How does the datset handle invalid datset ?
* What do we want to do with null values ?
* Do we want to summarise, group or filter the data ?

## Import libraries and packages

In [1]:
import pandas as pd
import numpy as np

filename = '/content/drive/MyDrive/Data Science/Statistical Analysis/Diabetes.csv'

## Load dataset

In [2]:
## mount the google drive to fetch the data
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_csv(filename)
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


From the above head display of the dataset we can see that one row in **SkinThickness** has **0** value. This we can assume that since it had no data, therefore it was filled with 0 instead of null-values.

In [5]:
df2 = df[["Glucose", "BMI", "Age", "Outcome"]]

In [6]:
df2.head()

,Glucose,BMI,Age,Outcome
0,148,33.6,50,1
1,85,26.6,31,0
2,183,23.3,32,1
3,89,28.1,21,0
4,137,43.1,33,1


In [7]:
df2.describe()

,Glucose,BMI,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000
mean,120.894531,31.992578,33.240885,0.348958
std,31.972618,7.884160,11.760232,0.476951
min,0.000000,0.000000,21.000000,0.000000
25%,99.000000,27.300000,24.000000,0.000000
50%,117.000000,32.000000,29.000000,0.000000
75%,140.250000,36.600000,41.000000,1.000000
max,199.000000,67.100000,81.000000,1.000000


The minimum values for **Glucose** and **BMI** are **0** which is probably not true. 

## Handle zero values

In [8]:
## Get the row values where it is 0 apart from 'Outcome' column
df3 = df2.loc[~(df2[df2.columns[: -1]] == 0).any(axis=1)]

In [9]:
df3.describe()
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 752 entries, 0 to 767
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Glucose  752 non-null    int64  
 1   BMI      752 non-null    float64
 2   Age      752 non-null    int64  
 3   Outcome  752 non-null    int64  
dtypes: float64(1), int64(3)
memory usage: 29.4 KB


In [10]:
## Handle '0' values with null values
df4 = df2.replace(0, np.nan)

In [11]:
df4.describe()

,Glucose,BMI,Age,Outcome
count,763.000000,757.000000,768.000000,268.0
mean,121.686763,32.457464,33.240885,1.0
std,30.535641,6.924988,11.760232,0.0
min,44.000000,18.200000,21.000000,1.0
25%,99.000000,27.500000,24.000000,1.0
50%,117.000000,32.300000,29.000000,1.0
75%,141.000000,36.600000,41.000000,1.0
max,199.000000,67.100000,81.000000,1.0


In [13]:
df4.isna().sum()

Glucose      5
BMI         11
Age          0
Outcome    500
dtype: int64

In the above code we replace **zero** values with null values since **null** values can be replced by mean, median, mode or any machine learning model prediction values.

## Format data on basis of outcome

In [14]:
df3.groupby('Outcome').mean()

,Glucose,BMI,Age
Outcome,,,
0,110.825820,30.876434,31.309426
1,142.488636,35.371970,37.015152


In [15]:
df.groupby('Outcome').agg({'Glucose':'mean', 'BMI':'median', 'Age':'sum'})

,Glucose,BMI,Age
Outcome,,,
0,109.980000,30.05,15595
1,141.257463,34.25,9934


Now we want to get multiple aggregates per column. We can pass a **list** to the groupby column. This will show how skew your data is.

In [16]:
df3.groupby('Outcome').agg(['mean', 'median'])

Glucose               BMI               Age       
               mean median       mean median       mean median
Outcome                                                       
0        110.825820  107.5  30.876434  30.10  31.309426   27.0
1        142.488636  140.5  35.371970  34.25  37.015152   36.0

## Split dataframe in multiple dataframes

In [17]:
positive = df3.loc[df3.Outcome == 1]
negative = df3.loc[df3.Outcome == 0]
print(positive.shape, negative.shape)

(264, 4) (488, 4)
